### Description:
In this notebook I will try to EDA the emails

### Import data and print some rows

In [2]:
import pandas as pd

df = pd.read_csv("enron_test.csv")
print("DF shape:", df.shape)
df.head()

DF shape: (1000, 6)


,Date,From,To,Subject,content,new_date
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Here is our forecast\n\n,2001-05-14 23:39:00
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Traveling to have a business meeting takes the...,2001-05-04 20:51:00
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,test successful. way to go!!!,2000-10-18 10:00:00
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,"Randy,\n\n Can you send me a schedule of the s...",2000-10-23 13:13:00
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Let's shoot for Tuesday at 11:45.,2000-08-31 12:07:00


In [3]:
# Convert emails to lists

def get_emails_as_list(e):
    if isinstance(e, str):
        return list(eval(e))
    return e

df['From'] = df['From'].apply(lambda x: get_emails_as_list(x))
df['To']   = df['To'].apply(lambda x: get_emails_as_list(x))
df.head()

,Date,From,To,Subject,content,new_date
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",[phillip.allen@enron.com],[tim.belden@enron.com],NaN,Here is our forecast\n\n,2001-05-14 23:39:00
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",[phillip.allen@enron.com],[john.lavorato@enron.com],Re:,Traveling to have a business meeting takes the...,2001-05-04 20:51:00
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",[phillip.allen@enron.com],[leah.arsdall@enron.com],Re: test,test successful. way to go!!!,2000-10-18 10:00:00
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",[phillip.allen@enron.com],[randall.gay@enron.com],NaN,"Randy,\n\n Can you send me a schedule of the s...",2000-10-23 13:13:00
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",[phillip.allen@enron.com],[greg.piper@enron.com],Re: Hello,Let's shoot for Tuesday at 11:45.,2000-08-31 12:07:00


In [4]:
# Check if there is more than one sender per email

count = 0
for e in df['From'].tolist():
    if len(e)>1:
        count+=1
print("%d emails with more than one sender" % count)

0 emails with more than one sender


In [5]:
# Check if there is more than one recipients per email
import math

count = 0
for e in df['To'].tolist():
    if isinstance(e, list) and len(e)>1:
        count+=1
print("%d emails with more than one recipient" % count)

90 emails with more than one recipient


### EDA sender emails

In [9]:
from collections import Counter

# Get list of sender emails
senders = [s[0] for s in df['From'].tolist()]

# Get unique sender emails and sort by their counts in descending order
senders_counter = Counter(senders)
ls = [(v,k) for k,v in senders_counter.most_common()]

print("Unique senders count:", len(ls))
ls[:10]

Unique senders count: 41


[(946, 'phillip.allen@enron.com'),
 (5, 'critical.notice@enron.com'),
 (4, 'ina.rangel@enron.com'),
 (2, 'messenger@ecm.bloomberg.com'),
 (2, 'aod@newsdata.com'),
 (2, 'webmaster@earnings.com'),
 (2, 'sarah.novosel@enron.com'),
 (2, 'ei_editor@ftenergy.com'),
 (2, 'announce@inbox.nytimes.com'),
 (2, 'jsmith@austintx.com')]

### Findings:
Looks like 'phillip.allen@enron.com' is the main sender

### EDA recipient emails

In [11]:
# Add all recipient emails into single list
recipients = []
for r in df['To'].tolist():
    if isinstance(r, list):
        recipients.extend(r)

# Get unique recipient emails and sort by their counts in descending order
recipient_counter = Counter(recipients)
ls = [(v,k) for k,v in recipient_counter.most_common()]

print("Unique recipients count:", len(ls))
ls[:15]

Unique recipients count: 307


[(68, 'keith.holst@enron.com'),
 (68, 'ina.rangel@enron.com'),
 (60, 'jsmith@austintx.com'),
 (60, 'mike.grigsby@enron.com'),
 (59, 'stagecoachmama@hotmail.com'),
 (42, 'pallen70@hotmail.com'),
 (35, 'frank.ermis@enron.com'),
 (31, 'maryrichards7@hotmail.com'),
 (29, 'matthew.lenhart@enron.com'),
 (27, 'john.lavorato@enron.com'),
 (24, 'cbpres@austin.rr.com'),
 (22, 'jacquestc@aol.com'),
 (17, 'jay.reitmeyer@enron.com'),
 (16, 'llewter@austin.rr.com'),
 (15, 'tim.belden@enron.com')]

In [68]:
# Check if the most frequent sender receives emails too
'phillip.allen@enron.com' in recipients

True

### Check the sender-recipient group counts

In [13]:
# Get sender-recipient pairs
send_rec = []
for idx, row in df[df.To.notnull()].iterrows():
    for r in row["To"]:
        send_rec.append((row["From"][0], r))
send_rec[:10]

[('phillip.allen@enron.com', 'tim.belden@enron.com'),
 ('phillip.allen@enron.com', 'john.lavorato@enron.com'),
 ('phillip.allen@enron.com', 'leah.arsdall@enron.com'),
 ('phillip.allen@enron.com', 'randall.gay@enron.com'),
 ('phillip.allen@enron.com', 'greg.piper@enron.com'),
 ('phillip.allen@enron.com', 'greg.piper@enron.com'),
 ('phillip.allen@enron.com', 'david.l.johnson@enron.com'),
 ('phillip.allen@enron.com', 'john.shafer@enron.com'),
 ('phillip.allen@enron.com', 'joyce.teixeira@enron.com'),
 ('phillip.allen@enron.com', 'mark.scott@enron.com')]

In [14]:
# Count sender-recipient pairs
ls = [(v,k) for k,v in Counter(send_rec).most_common()]
ls

[(68, ('phillip.allen@enron.com', 'keith.holst@enron.com')),
 (68, ('phillip.allen@enron.com', 'ina.rangel@enron.com')),
 (60, ('phillip.allen@enron.com', 'jsmith@austintx.com')),
 (59, ('phillip.allen@enron.com', 'stagecoachmama@hotmail.com')),
 (55, ('phillip.allen@enron.com', 'mike.grigsby@enron.com')),
 (42, ('phillip.allen@enron.com', 'pallen70@hotmail.com')),
 (35, ('phillip.allen@enron.com', 'frank.ermis@enron.com')),
 (31, ('phillip.allen@enron.com', 'maryrichards7@hotmail.com')),
 (29, ('phillip.allen@enron.com', 'matthew.lenhart@enron.com')),
 (26, ('phillip.allen@enron.com', 'john.lavorato@enron.com')),
 (24, ('phillip.allen@enron.com', 'cbpres@austin.rr.com')),
 (22, ('phillip.allen@enron.com', 'jacquestc@aol.com')),
 (16, ('phillip.allen@enron.com', 'jay.reitmeyer@enron.com')),
 (16, ('phillip.allen@enron.com', 'llewter@austin.rr.com')),
 (15, ('phillip.allen@enron.com', 'tim.belden@enron.com')),
 (15, ('phillip.allen@enron.com', 'jeff.richter@enron.com')),
 (14, ('phillip